#### Import

In [ ]:
# Core & preprocessing
import pandas as pd
import numpy as np
from scipy.sparse import hstack

# Feature processing
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, MinMaxScaler
from sklearn.feature_extraction.text import TfidfVectorizer

# Models
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from xgboost import XGBClassifier

# Evaluation
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV, StratifiedKFold


#### Data Preprocessing

In [ ]:
# Load data
columns = [
    "label", "statement", "subject", "speaker", "speaker_job",
    "state_info", "party_affiliation", "barely_true_counts", "false_counts",
    "half_true_counts", "mostly_true_counts", "pants_on_fire_counts", "context"
]
train_df = pd.read_csv("train.tsv", sep='\t', header=None, names=columns)
valid_df = pd.read_csv("valid.tsv", sep='\t', header=None, names=columns)
test_df  = pd.read_csv("test.tsv",  sep='\t', header=None, names=columns)

In [ ]:
# Encode labels
label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(train_df['label'])
y_valid = label_encoder.transform(valid_df['label'])
y_test  = label_encoder.transform(test_df['label'])

In [ ]:
# TF-IDF
tfidf = TfidfVectorizer(ngram_range=(1,2), max_features=5000)
X_train_text = tfidf.fit_transform(train_df['statement'])
X_valid_text = tfidf.transform(valid_df['statement'])
X_test_text  = tfidf.transform(test_df['statement'])

In [ ]:
# Metadata (categorical)
meta_cols = ['party_affiliation', 'speaker_job']
encoder = OneHotEncoder(handle_unknown='ignore')
train_meta = encoder.fit_transform(train_df[meta_cols])
valid_meta = encoder.transform(valid_df[meta_cols])
test_meta  = encoder.transform(test_df[meta_cols])

In [ ]:
# Credit history fix
credit_cols = ["pants_on_fire_counts", "false_counts", "barely_true_counts", "half_true_counts", "mostly_true_counts"]
for df in [train_df, valid_df, test_df]:
    df[credit_cols] = df[credit_cols].fillna(0).astype(int)

def correct_credit(row):
    counts = row[credit_cols].values.astype(int).copy()
    label = row['label']
    if label == "pants-fire": counts[0] = max(0, counts[0] - 1)
    elif label == "false": counts[1] = max(0, counts[1] - 1)
    elif label == "barely-true": counts[2] = max(0, counts[2] - 1)
    elif label == "half-true": counts[3] = max(0, counts[3] - 1)
    elif label == "mostly-true": counts[4] = max(0, counts[4] - 1)
    return counts

train_df['credit_vector'] = train_df.apply(correct_credit, axis=1)
valid_df['credit_vector'] = valid_df.apply(correct_credit, axis=1)
test_df['credit_vector']  = test_df.apply(correct_credit, axis=1)

credit_train = np.stack(train_df['credit_vector'].values).astype(float)
credit_valid = np.stack(valid_df['credit_vector'].values).astype(float)
credit_test  = np.stack(test_df['credit_vector'].values).astype(float)

In [ ]:
scaler = MinMaxScaler()
credit_train = scaler.fit_transform(credit_train)
credit_valid = scaler.transform(credit_valid)
credit_test  = scaler.transform(credit_test)

# Final features
X_train_ext = hstack([X_train_text, train_meta, credit_train])
X_valid_ext = hstack([X_valid_text, valid_meta, credit_valid])
X_test_ext  = hstack([X_test_text,  test_meta,  credit_test])


#### Model Training

In [ ]:
lr_model = LogisticRegression(max_iter=1000)
lr_model.fit(X_train_ext, y_train)
val_preds = lr_model.predict(X_valid_ext)
print("Logistic Regression:\n", accuracy_score(y_valid, val_preds))
print(classification_report(y_valid, val_preds, target_names=label_encoder.classes_))


Logistic Regression:
 0.2733644859813084
              precision    recall  f1-score   support

 barely-true       0.22      0.17      0.19       237
       false       0.29      0.32      0.31       263
   half-true       0.25      0.29      0.27       248
 mostly-true       0.30      0.31      0.30       251
  pants-fire       0.44      0.29      0.35       116
        true       0.23      0.26      0.24       169

    accuracy                           0.27      1284
   macro avg       0.29      0.27      0.28      1284
weighted avg       0.28      0.27      0.27      1284



In [ ]:
nb_model = MultinomialNB()
nb_model.fit(X_train_text, y_train)
val_preds = nb_model.predict(X_valid_text)
print("Naive Bayes:\n", accuracy_score(y_valid, val_preds))
print(classification_report(y_valid, val_preds, target_names=label_encoder.classes_))

Naive Bayes:
 0.25
              precision    recall  f1-score   support

 barely-true       0.21      0.11      0.14       237
       false       0.26      0.38      0.31       263
   half-true       0.23      0.37      0.29       248
 mostly-true       0.28      0.31      0.30       251
  pants-fire       0.25      0.01      0.02       116
        true       0.23      0.15      0.18       169

    accuracy                           0.25      1284
   macro avg       0.24      0.22      0.21      1284
weighted avg       0.25      0.25      0.23      1284



In [ ]:
svm_model = SVC(kernel='rbf')
svm_model.fit(X_train_ext, y_train)
val_preds = svm_model.predict(X_valid_ext)
print("SVM:\n", accuracy_score(y_valid, val_preds))
print(classification_report(y_valid, val_preds, target_names=label_encoder.classes_))


SVM:
 0.2772585669781931
              precision    recall  f1-score   support

 barely-true       0.28      0.13      0.18       237
       false       0.30      0.38      0.34       263
   half-true       0.23      0.33      0.27       248
 mostly-true       0.26      0.37      0.31       251
  pants-fire       0.64      0.24      0.35       116
        true       0.25      0.13      0.17       169

    accuracy                           0.28      1284
   macro avg       0.33      0.26      0.27      1284
weighted avg       0.30      0.28      0.27      1284



In [ ]:
knn_model = KNeighborsClassifier(n_neighbors=5)
knn_model.fit(X_train_ext, y_train)
val_preds = knn_model.predict(X_valid_ext)
print("KNN:\n", accuracy_score(y_valid, val_preds))
print(classification_report(y_valid, val_preds, target_names=label_encoder.classes_))

KNN:
 0.2235202492211838
              precision    recall  f1-score   support

 barely-true       0.22      0.28      0.25       237
       false       0.24      0.24      0.24       263
   half-true       0.20      0.22      0.21       248
 mostly-true       0.23      0.21      0.22       251
  pants-fire       0.35      0.25      0.29       116
        true       0.16      0.12      0.14       169

    accuracy                           0.22      1284
   macro avg       0.23      0.22      0.22      1284
weighted avg       0.23      0.22      0.22      1284



In [ ]:
dt_model = DecisionTreeClassifier()
dt_model.fit(X_train_ext, y_train)
val_preds = dt_model.predict(X_valid_ext)
print("Decision Tree:\n", accuracy_score(y_valid, val_preds))
print(classification_report(y_valid, val_preds, target_names=label_encoder.classes_))


Decision Tree:
 0.4462616822429907
              precision    recall  f1-score   support

 barely-true       0.47      0.43      0.45       237
       false       0.47      0.48      0.47       263
   half-true       0.42      0.44      0.43       248
 mostly-true       0.47      0.43      0.45       251
  pants-fire       0.60      0.53      0.57       116
        true       0.33      0.40      0.36       169

    accuracy                           0.45      1284
   macro avg       0.46      0.45      0.45      1284
weighted avg       0.45      0.45      0.45      1284



In [ ]:
rf_model = RandomForestClassifier()
rf_model.fit(X_train_ext, y_train)
val_preds = rf_model.predict(X_valid_ext)
print("Random Forest:\n", accuracy_score(y_valid, val_preds))
print(classification_report(y_valid, val_preds, target_names=label_encoder.classes_))


Random Forest:
 0.3847352024922118
              precision    recall  f1-score   support

 barely-true       0.43      0.27      0.33       237
       false       0.37      0.50      0.43       263
   half-true       0.34      0.46      0.39       248
 mostly-true       0.37      0.43      0.40       251
  pants-fire       0.73      0.34      0.47       116
        true       0.37      0.22      0.27       169

    accuracy                           0.38      1284
   macro avg       0.43      0.37      0.38      1284
weighted avg       0.41      0.38      0.38      1284



In [ ]:
ada_model = AdaBoostClassifier()
ada_model.fit(X_train_ext, y_train)
val_preds = ada_model.predict(X_valid_ext)
print("AdaBoost:\n", accuracy_score(y_valid, val_preds))
print(classification_report(y_valid, val_preds, target_names=label_encoder.classes_))


AdaBoost:
 0.27102803738317754
              precision    recall  f1-score   support

 barely-true       0.00      0.00      0.00       237
       false       0.26      0.44      0.33       263
   half-true       0.23      0.54      0.32       248
 mostly-true       0.37      0.23      0.28       251
  pants-fire       0.68      0.22      0.33       116
        true       0.29      0.08      0.13       169

    accuracy                           0.27      1284
   macro avg       0.30      0.25      0.23      1284
weighted avg       0.27      0.27      0.23      1284



/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
xgb_model = XGBClassifier()
xgb_model.fit(X_train_ext, y_train)
val_preds = xgb_model.predict(X_valid_ext)
print("XGBoost:\n", accuracy_score(y_valid, val_preds))
print(classification_report(y_valid, val_preds, target_names=label_encoder.classes_))


XGBoost:
 0.3956386292834891
              precision    recall  f1-score   support

 barely-true       0.39      0.28      0.33       237
       false       0.39      0.43      0.41       263
   half-true       0.38      0.47      0.42       248
 mostly-true       0.39      0.45      0.42       251
  pants-fire       0.66      0.39      0.49       116
        true       0.35      0.33      0.34       169

    accuracy                           0.40      1284
   macro avg       0.43      0.39      0.40      1284
weighted avg       0.41      0.40      0.39      1284



#### Hyper Parameter Tuning

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import RandomizedSearchCV
import numpy as np

# Define parameter grid
lr_params = {
    'C': [0.01, 0.1, 1, 10],
    'penalty': ['l1', 'l2'],
    'solver': ['liblinear']
}

# Some combinations are invalid → filter them with error_score='raise' for clarity
lr_random = RandomizedSearchCV(
    estimator=LogisticRegression(max_iter=2000),
    param_distributions=lr_params,
    n_iter=20,
    cv=5,
    scoring='accuracy',
    n_jobs=-1,
    random_state=42,
    error_score='raise'  # this will show exact issues if invalid params are picked
)

# Fit
lr_random.fit(X_train_ext, y_train)

# Output results
print("🔍 Logistic Regression Best Params:", lr_random.best_params_)
print("Validation Accuracy:", lr_random.score(X_valid_ext, y_valid))

/usr/local/lib/python3.11/dist-packages/sklearn/model_selection/_search.py:317: UserWarning: The total space of parameters 8 is smaller than n_iter=20. Running 8 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


🔍 Logistic Regression Best Params: {'solver': 'liblinear', 'penalty': 'l2', 'C': 1}
Validation Accuracy: 0.2803738317757009


In [ ]:
nb_params = {
    'alpha': np.linspace(0.01, 10.0, 20),
    'fit_prior': [True, False]
}

nb_random = RandomizedSearchCV(MultinomialNB(), nb_params, n_iter=20, cv=5, scoring='accuracy', n_jobs=-1, random_state=42)
nb_random.fit(X_train_text, y_train)

print("🔍 Naive Bayes Best Params:", nb_random.best_params_)
print("Validation Accuracy:", nb_random.score(X_valid_text, y_valid))


🔍 Naive Bayes Best Params: {'fit_prior': False, 'alpha': np.float64(2.113157894736842)}
Validation Accuracy: 0.25934579439252337


In [ ]:
from sklearn.svm import SVC
from sklearn.model_selection import RandomizedSearchCV

svm_params = {
    'C': [0.01, 0.1, 1, 10],
    'kernel': ['linear', 'rbf', 'poly'],
    'gamma': ['scale', 'auto']
}

svm_random = RandomizedSearchCV(SVC(), svm_params, n_iter=10, cv=5, scoring='accuracy', n_jobs=-1, random_state=42)
svm_random.fit(X_train_ext, y_train)

print("🔍 SVC Best Params:", svm_random.best_params_)
print("Validation Accuracy:", svm_random.score(X_valid_ext, y_valid))

🔍 SVC Best Params: {'kernel': 'rbf', 'gamma': 'scale', 'C': 1}
Validation Accuracy: 0.2749221183800623


In [ ]:
knn_params = {
    'n_neighbors': list(range(3, 30, 2)),
    'weights': ['uniform', 'distance'],
    'metric': ['euclidean', 'manhattan', 'minkowski']
}

knn_random = RandomizedSearchCV(KNeighborsClassifier(), knn_params, n_iter=20, cv=5, scoring='accuracy', n_jobs=-1, random_state=42)
knn_random.fit(X_train_ext, y_train)

print("🔍 KNN Best Params:", knn_random.best_params_)
print("Validation Accuracy:", knn_random.score(X_valid_ext, y_valid))


🔍 KNN Best Params: {'weights': 'distance', 'n_neighbors': 27, 'metric': 'minkowski'}
Validation Accuracy: 0.26791277258566976


In [ ]:
dt_params = {
    'max_depth': [10, 20, 30, 40, 50, None],
    'min_samples_split': [2, 5, 10, 20],
    'min_samples_leaf': [1, 2, 4, 10],
    'criterion': ['gini', 'entropy', 'log_loss']
}

dt_random = RandomizedSearchCV(DecisionTreeClassifier(random_state=42), dt_params, n_iter=20, cv=5, scoring='accuracy', n_jobs=-1, random_state=42)
dt_random.fit(X_train_ext, y_train)

print("🔍 Decision Tree Best Params:", dt_random.best_params_)
print("Validation Accuracy:", dt_random.score(X_valid_ext, y_valid))


🔍 Decision Tree Best Params: {'min_samples_split': 2, 'min_samples_leaf': 1, 'max_depth': 50, 'criterion': 'log_loss'}
Validation Accuracy: 0.4719626168224299


In [ ]:
rf_params = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2],
    'bootstrap': [True, False]
}

rf_random = RandomizedSearchCV(RandomForestClassifier(random_state=42), rf_params, n_iter=30, cv=5, scoring='accuracy', n_jobs=-1, random_state=42)
rf_random.fit(X_train_ext, y_train)

print("🔍 Random Forest Best Params:", rf_random.best_params_)
print("Validation Accuracy:", rf_random.score(X_valid_ext, y_valid))


🔍 Random Forest Best Params: {'n_estimators': 100, 'min_samples_split': 2, 'min_samples_leaf': 1, 'max_depth': None, 'bootstrap': True}
Validation Accuracy: 0.3808411214953271


In [ ]:
ada_params = {
    'n_estimators': [50, 100, 200],
    'learning_rate': [0.01, 0.1, 1.0],
    'algorithm': ['SAMME', 'SAMME.R']
}

ada_random = RandomizedSearchCV(AdaBoostClassifier(random_state=42), ada_params, n_iter=20, cv=5, scoring='accuracy', n_jobs=-1, random_state=42)
ada_random.fit(X_train_ext, y_train)

print("🔍 AdaBoost Best Params:", ada_random.best_params_)
print("Validation Accuracy:", ada_random.score(X_valid_ext, y_valid))


/usr/local/lib/python3.11/dist-packages/sklearn/model_selection/_search.py:317: UserWarning: The total space of parameters 18 is smaller than n_iter=20. Running 18 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/model_selection/_validation.py:528: FitFailedWarning: 
45 fits failed out of a total of 90.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
45 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/sklearn/model_selection/_validation.py", line 866, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.11/dist-packages/sklearn/base.py", line 1382, in 

🔍 AdaBoost Best Params: {'n_estimators': 200, 'learning_rate': 1.0, 'algorithm': 'SAMME'}
Validation Accuracy: 0.29361370716510904


In [ ]:
xgb_params = {
    'n_estimators': [100, 200],
    'learning_rate': [0.01, 0.1, 0.3],
    'max_depth': [3, 6, 10],
    'subsample': [0.7, 1.0],
    'colsample_bytree': [0.7, 1.0]
}

xgb_random = RandomizedSearchCV(
    XGBClassifier(use_label_encoder=False, objective='multi:softmax', num_class=6, eval_metric='mlogloss', random_state=42),
    xgb_params,
    n_iter=30,
    cv=5,
    scoring='accuracy',
    n_jobs=-1,
    random_state=42
)

xgb_random.fit(X_train_ext, y_train)
print("🔍 XGBoost Best Params:", xgb_random.best_params_)
print("Validation Accuracy:", xgb_random.score(X_valid_ext, y_valid))


/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [03:05:47] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


🔍 XGBoost Best Params: {'subsample': 1.0, 'n_estimators': 200, 'max_depth': 10, 'learning_rate': 0.1, 'colsample_bytree': 1.0}
Validation Accuracy: 0.43613707165109034
